In [ ]:
import os
import re
import torch
import difflib
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

def anonymizar_codigo(code):
    tokens = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', code)
    usados = {}
    nuevo_codigo = code
    contador = 1
    for tok in tokens:
        if tok not in usados and tok not in {"def", "if", "else", "for", "while", "return", "print", "input"}:
            usados[tok] = f"VAR_{contador}"
            contador += 1
    for original, nuevo in usados.items():
        nuevo_codigo = re.sub(rf'\b{original}\b', nuevo, nuevo_codigo)
    return nuevo_codigo

def extraer_funciones(code):
    patron = r"(def\s+[a-zA-Z_][a-zA-Z0-9_]*\(.*?\):(?:\n(?:\s{4}|\t).*)*)"
    return re.findall(patron, code)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

def cargar_funciones_por_archivo(carpeta):
    base = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".py"):
            with open(os.path.join(carpeta, archivo), "r", encoding="utf-8", errors="ignore") as f:
                funciones = extraer_funciones(f.read())
                for idx, func in enumerate(funciones):
                    if len(func.splitlines()) >= 5:
                        base.append({
                            "archivo": archivo,
                            "funcion": f"{archivo}::func{idx+1}",
                            "codigo": func,
                            "anonimizado": anonymizar_codigo(func)
                        })
    return base

def comparar_funciones_y_exportar(carpeta, umbral=0.8):
    base = cargar_funciones_por_archivo(carpeta)
    embeddings = [get_embedding(entry["anonimizado"]) for entry in base]
    resultados = []

    for i in range(len(base)):
        for j in range(i+1, len(base)):
            if base[i]["archivo"] == base[j]["archivo"]:
                continue
            code1, code2 = base[i]["codigo"], base[j]["codigo"]
            emb_sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            txt_sim = difflib.SequenceMatcher(None, code1, code2).ratio()
            final_score = 0.7 * emb_sim + 0.3 * txt_sim

            if final_score >= umbral:
                resultados.append({
                    "funcion_1": base[i]["funcion"],
                    "funcion_2": base[j]["funcion"],
                    "archivo_1": base[i]["archivo"],
                    "archivo_2": base[j]["archivo"],
                    "similitud_CodeBERT": round(emb_sim, 4),
                    "similitud_textual": round(txt_sim, 4),
                    "score_final": round(final_score, 4)
                })

    df = pd.DataFrame(resultados)
    df.to_excel("resultados_plagio_hibrido.xlsx", index=False)
    print("Resultados guardados en resultados_plagio_hibrido.xlsx")

# Usa tu propia ruta aquí
comparar_funciones_y_exportar("dataset")


Resultados guardados en resultados_plagio_hibrido.xlsx
